The experiment of BBI onto MNIST dataset with different architectures.

In [13]:
import torch 
import torch.nn as nn
from torchvision import datasets, transforms
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('mnist/', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('mnist/', train=False, download=True, transform=transform)

batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

In [17]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features=64 * 3 * 3, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=10)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = torch.flatten(x, start_dim=1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

With Adam optimizer at first.

In [20]:
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train
epochs = 3
for epoch in range(epochs):
    loss_tr = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        loss_tr += loss.item()
    print(f'Epoch {epoch + 1}, avrg. loss: ', loss_tr / len(train_loader))

# Evaluate
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: ', (100 * correct / total))

Epoch [1], loss: 0.2147
Epoch [2], loss: 0.0574
Epoch [3], loss: 0.0399
Accuracy of the network on the 10000 test images: 98 %


In [26]:
# Grid Search
learning_rates = [0.01, 0.001, 0.0001]
weight_decay_values = [0, 0.1, 0.3]
batch_sizes = [32, 64, 128]
num_epochs = 3

hyperparameters = [(lr, wdv, bs) for lr in learning_rates for wdv in weight_decay_values for bs in batch_sizes]
for lr, wdv, bs in hyperparameters:
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=bs, shuffle=True, num_workers=2)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=bs, shuffle=True, num_workers=2)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wdv)
    
    for epoch in range(num_epochs):
        for i, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
    accuracy = 100 * correct / total
    print(f'Learning rate: {lr}, weight decay: {wdv}, batch size: {bs}, epoch: {epoch}, accuracy: {accuracy}')

Learning rate: 0.01, weight decay: 0, batch size: 32, epoch: 2, accuracy: 95.47
Learning rate: 0.01, weight decay: 0, batch size: 64, epoch: 2, accuracy: 96.34
Learning rate: 0.01, weight decay: 0, batch size: 128, epoch: 2, accuracy: 97.43
Learning rate: 0.01, weight decay: 0.1, batch size: 32, epoch: 2, accuracy: 92.62
Learning rate: 0.01, weight decay: 0.1, batch size: 64, epoch: 2, accuracy: 92.73
Learning rate: 0.01, weight decay: 0.1, batch size: 128, epoch: 2, accuracy: 91.31
